# Benchmarking Test File

Use this notebook to compare a run of SPECTRA with the benchmark data. 

The data you're comparing is in the folder marked "benchmark_factors" on the google drive. There is one csv for each factor, and each file is named the same as the factor you should be comparing it against.

There are two main things we're looking at - pairwise correlation between each factor and the intersection of the genes that contribute to each factor.

In [2]:
# imports
import re
import scanpy as sc
import seaborn
import numpy as np
import csv

This is all buggy right now - mainly waiting to see how to match factor weights back to genes, and waiting to see if a new run returns each factor individually like this, or if we have to pull that apart ourselves.

In [ ]:
# Importing files

# new run - remember to change this to be your local file path
adata = sc.read_h5ad('C:\\Users\\phill\\Documents\\HMCFall22\\Clinic\\data_for_clinic_2023.h5ad')

# grabbing the list of factors from the new data
def match(input_string):
    return re.findall(r'[\d]*-X-[\w_-]*X-norm', input_string)
    
factor_matches = [match(word) for word in adata.obs]

factor_list = []

for l in factor_matches:
    if l:
        factor_list.append(l[0])


In [1]:
# Intersection between factors
# for now assuming the anndata looks identical to the benchmark set we recieved.

for l in factor_list:
    benchmark_factor = []
    
    # grab the benchmark data
    with open('{l}.csv', mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)

        # get list of factors from csv
        factors_1 = list(csv_reader)

        csv_file.close()

    print(benchmark_factor)

IndentationError: expected an indented block (3081531705.py, line 5)

In [ ]:
# Pairwise correlation - buggy
corr_mat = np.corrcoef(num_factors_1, num_factors_2)

seaborn.clustermap(corr_mat)